In [ ]:
import pandas as pd

In [ ]:
filename = 'train.txt'
user_list = [] # 记录已经存在的用户关系
user_list_clean = [] # 记录在数据中出现的用户
with open(filename, 'r') as file_to_read:
  while True:
    lines = file_to_read.readline() # 整行读取数据
    if not lines:
      break
      pass
    user_list.append([i.replace("\n", "") for i in lines.split("\t")])
#     user_list_clean.extend([i.replace("\n", "") for i in lines.split("\t")])
    pass
# user_list_clean = set(user_list_clean) #去重

In [ ]:
road_edges_exist = [] # label为1的用户组，指它们之间是有关系的（from，to, label = 1），并且接下来要用所有存在的边来生成用户图谱
for line in user_list:
    way_out = line[0] # way_out指的是根节点用户
    if len(line) > 1: # 数据中有一个人是没有好友的，所以要防止报错
        for i in line[1:]:
            road_edges_exist.append((way_out, i, 1))
    else:
        continue
road_edges_exist = list(set(road_edges_exist)) #去重（有几十万条重复项）

In [ ]:
len(road_edges_exist)

In [ ]:
# 生成label为0的训练集，预计花费30小时
import random
road_edges_non_exist = [] # label为0的用户组，指它们之间是没有关系的（from，to, label = 0）
count = 0
for user in user_list_clean:
    us_from = user
    us_to = random.choice(user_list_clean) # 需要打开第二代码块中的注释
    if (us_from != us_to) and ((us_from, us_to, 1) not in road_edges_exist):
        road_edges_non_exist.append((us_from, us_to, 0))
        count += 1
    if count == 25000:
        break

In [ ]:
# 选择25000个随机的label等于1的数据放入训练数据集
from random import sample
data_label_1 = sample(road_edges_exist, 25000)

In [ ]:
# 构建图结构
from igraph import Graph as IGraph

In [ ]:
g = IGraph.TupleList(road_edges_exist, directed=True, vertex_name_attr='name', edge_attrs=None, weights=True)
names = g.vs["name"]
weights = g.es["weight"] #权重是1，就是不考虑权重影响
g.is_weighted()

In [ ]:
# 以下是全样本分析
df = pd.read_csv("train_data.csv")
df

In [ ]:
# 迭代dataframe太慢了，还是迭代列表快一些。每次执行剩下操作前都先读取到from和to列表。如果要直接迭代dataframe，就用iterrows()
from_list = []
to_list = []

for index, row in df.iterrows():
    from_list.append(str(row["from"]))
    to_list.append(str(row["to"]))

In [ ]:
# 图节点的度（八个指标，出入度各四个）(全样本分析)
in_degree_from = []
in_degree_to = []
out_degree_from = []
out_degree_to = []
overall_degree_from = []
overall_degree_to = []
bi_degree_from = []
bi_degree_to = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    in_degree_from.append(g.indegree(from_list[index]))
    in_degree_to.append(g.indegree(to_list[index]))
    out_degree_from.append(g.outdegree(from_list[index]))
    out_degree_to.append(g.outdegree(to_list[index]))
    overall_degree_from.append(g.degree(from_list[index]))
    overall_degree_to.append(g.degree(to_list[index]))
    bi_degree_from.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(from_list[index], "out")))))
    bi_degree_to.append(len(set(g.neighbors(to_list[index], "in")).intersection(set(g.neighbors(to_list[index], "out")))))
    if index % 1000 == 0:
        print("complete {}".format(index))

In [ ]:
# 将新的feature放入原先测试数据集中
df['in_degree_from'] = in_degree_from
df['in_degree_to'] = in_degree_to
df['out_degree_from'] = out_degree_from
df['out_degree_to'] = out_degree_to
df['overall_degree_from'] = overall_degree_from
df['overall_degree_to'] = overall_degree_to
df['bi_degree_from'] = bi_degree_from
df['bi_degree_to'] = bi_degree_to

In [ ]:
# common_friends feature construct(这三个指标，真·废到家了)
in_common_friends = []
out_common_friends = []
bi_common_friends = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    in_common_friends.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(to_list[index], "in")))))
    out_common_friends.append(len(set(g.neighbors(from_list[index], "out")).intersection(set(g.neighbors(to_list[index], "out")))))
    bi_common_friends.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(from_list[index], "out"))).intersection(set(g.neighbors(to_list[index], "in")).intersection(set(g.neighbors(to_list[index], "out"))))))
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
df['in_common_friends'] = in_common_friends
df['out_common_friends'] = out_common_friends
df['bi_common_friends'] = bi_common_friends

In [ ]:
# total_friends feature construct
in_total_friends = []
out_total_friends = []
bi_total_friends = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    in_total_friends.append(len(set(g.neighbors(from_list[index], "in")).union(set(g.neighbors(to_list[index], "in")))))
    out_total_friends.append(len(set(g.neighbors(from_list[index], "out")).union(set(g.neighbors(to_list[index], "out")))))
    bi_total_friends.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(from_list[index], "out"))).union(set(g.neighbors(to_list[index], "in")).intersection(set(g.neighbors(to_list[index], "out"))))))
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
df['in_total_friends'] = in_total_friends
df['out_total_friends'] = out_total_friends
df['bi_total_friends'] = bi_total_friends

In [ ]:
# transitive_friends feature construct
transitive_friends = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    transitive_friends.append(len(set(g.neighbors(from_list[index], "out")).intersection(set(g.neighbors(to_list[index], "in")))))
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
# preference_attanchment_score feature construct
preference_attanchment_score = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    preference_attanchment_score.append(g.degree(from_list[index]) * g.degree(to_list[index]))
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
df['transitive_friends'] = transitive_friends
df['preference_attanchment_score'] = preference_attanchment_score

In [ ]:
# 以下生成AA特征
import math

adamic_adar = []
# adamic_adar_out_in = []

length = len(from_list)
for index in range(length):
    from_intersection_to = list(set(g.neighbors(from_list[index])).intersection(set(g.neighbors(to_list[index]))))
    log_from_intersection_to = 0
    if len(from_intersection_to) > 0:
        for i in from_intersection_to:
            degree_i = g.degree(i)
            if degree_i > 1: #满足对数的真数大于0，且分母不等于0
                log_from_intersection_to += 1 / math.log(degree_i) # 根据其他文献，这里应该用以e为底的ln

    adamic_adar.append(log_from_intersection_to)
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
import math

# adamic_adar = []
adamic_adar_out_in = []

length = len(from_list)
for index in range(length):
    from_intersection_to = list(set(g.neighbors(from_list[index], "out")).intersection(set(g.neighbors(to_list[index], "in"))))
    log_from_intersection_to = 0
    if len(from_intersection_to) > 0:
        for i in from_intersection_to:
            degree_i = g.degree(i)
            if degree_i > 1: #满足对数的真数大于0，且分母不等于0
                log_from_intersection_to += 1 / math.log(degree_i) # 根据其他文献，这里应该用以e为底的ln

    adamic_adar_out_in.append(log_from_intersection_to)
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
df['adamic_adar'] = adamic_adar
df['adamic_adar_out_in'] = adamic_adar_out_in

In [ ]:
# friend-measure feature
friend_measure = []

length = len(from_list)
for index in range(length):
    u = from_list[index]
    v = to_list[index]
    u_nei = g.neighbors(u)
    v_nei = g.neighbors(v)
    name_u_nei = []
    name_v_nei = []
    for u_n in u_nei:
        name_u_nei.append(names[u_n])
    for v_n in v_nei:
        name_v_nei.append(names[v_n])
    
    measure_score = 0
    for u_n in name_u_nei:
        for v_n in name_v_nei:
#             if u_n == v_n or (u_n,v_n,1) in road_edges_exist or (v_n,u_n,1) in road_edges_exist:
# are_connected(u,v) Decides whether two given vertices are directly connected. (有向图中，该方法还会考虑方向，只有正向连接才会返回true)
            if u_n == v_n or g.are_connected(u_n, v_n) or g.are_connected(v_n, u_n): 
                measure_score += 1
            
    friend_measure.append(measure_score)
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
# oppsite direction friends features
oppsite_direction_friends = [] # 二值特征，如果（to，from）在链路中，那么from有很大可能关注to

length = len(from_list)
for index in range(length):
    u = from_list[index] # from
    v = to_list[index] # to
    if g.are_connected(v, u):
        oppsite_direction_friends.append(1)
    else:
        oppsite_direction_friends.append(0)
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
# subgragh edge count feature
subgragh_edge_count = []

length = len(from_list)
for index in range(length):
    u = from_list[index] # from
    v = to_list[index] # to
    from_union_to = list(set(g.neighbors(u)).union(set(g.neighbors(v))))
    
    # neighbors 返回的数不是节点的名字而是id，id是系统按顺序生成的，所以后面查看是否在链路中的时候要先用names[id]转换成name，再检查
    name_from_union_to = []
    for i in from_union_to:
        name_from_union_to.append(names[i])
    
    measure_score = 0
    for i in name_from_union_to:
        for j in name_from_union_to:
            if i == j:
                continue
            if g.are_connected(i, j):
                measure_score += 1
            else:
                continue
    subgragh_edge_count.append(measure_score)
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
# subgragh add edge count feature
subgragh_add_edge_count = []

length = len(from_list)
for index in range(length):
    u = from_list[index] # from
    v = to_list[index] # to
    from_union_to = list(set(g.neighbors(u)).union(set(g.neighbors(v)))) 
    # neighbors 返回的数不是节点的名字而是id，id是系统按顺序生成的，所以后面查看是否在链路中的时候要先用names[id]转换成name，再检查
    name_from_union_to = []
    for i in from_union_to:
        name_from_union_to.append(names[i])
    if u not in name_from_union_to:
        name_from_union_to.append(u)
    if v not in name_from_union_to:
        name_from_union_to.append(v)
    
    measure_score = 0
    for i in name_from_union_to:
        for j in name_from_union_to:
            if i == j:
                continue
            if g.are_connected(i, j):
                measure_score += 1
            else:
                continue
    subgragh_add_edge_count.append(measure_score)
    
    if index % 2000 == 0:
        print("completed {}".format(index))

In [ ]:
df['friend_measure'] = friend_measure
df['oppsite_direction_friends'] = oppsite_direction_friends
df['subgragh_edge_count'] = subgragh_edge_count
df['subgragh_add_edge_count'] = subgragh_add_edge_count

In [ ]:
df.to_csv("train_data.csv", index = False, header=True)

In [ ]:
# build RF classifier

# reshape train set
X = df.drop(["label", "from", "to"], axis = True)
y = df["label"]

In [ ]:
# split train and valid set
from sklearn import model_selection
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier # 随机森林
from sklearn.model_selection import GridSearchCV # gridsearchCV找到最优参数，参数调优优先考虑n_estimator（1-200）,即随机树的个数；之后考虑max_features即最大特征数1-20

param_test1 = {"n_estimators":range(1,101,10)}
RF_best = GridSearchCV(estimator=RandomForestClassifier(),param_grid=param_test1, scoring='roc_auc', cv=10)
RF_best.fit(X_train,y_train)

In [ ]:
RF_best.best_params_
RF_best.best_score_

In [ ]:
# build model using best parameters
RF_now = RandomForestClassifier(n_estimators=91)
RF_now.fit(X_train,y_train)

In [ ]:
# test AUC on valid set
from sklearn.metrics import roc_auc_score
# import numpy as np

y_valid_test = RF_now.predict(X_valid)
roc_auc_score(y_valid, y_valid_test)

In [ ]:
# 以下处理测试样本集
df_test = pd.read_csv("test-public.txt", sep='\t')
df_test = df_test.drop("Id", axis = 1)

In [ ]:
# rename
df_test.columns = ['from','to']

In [ ]:
from_list = []
to_list = []

for index, row in df_test.iterrows():
    from_list.append(str(row["from"]))
    to_list.append(str(row["to"]))

In [ ]:
# 图节点的度（八个指标，出入度各四个）(test set)
in_degree_from = []
in_degree_to = []
out_degree_from = []
out_degree_to = []
overall_degree_from = []
overall_degree_to = []
bi_degree_from = []
bi_degree_to = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    in_degree_from.append(g.indegree(from_list[index]))
    in_degree_to.append(g.indegree(to_list[index]))
    out_degree_from.append(g.outdegree(from_list[index]))
    out_degree_to.append(g.outdegree(to_list[index]))
    overall_degree_from.append(g.degree(from_list[index]))
    overall_degree_to.append(g.degree(to_list[index]))
    bi_degree_from.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(from_list[index], "out")))))
    bi_degree_to.append(len(set(g.neighbors(to_list[index], "in")).intersection(set(g.neighbors(to_list[index], "out")))))
    if (index+1) % 500 == 0:
        print("complete {}".format(index+1))

In [ ]:
# 将新的feature放入原先测试数据集中
df_test['in_degree_from'] = in_degree_from
df_test['in_degree_to'] = in_degree_to
df_test['out_degree_from'] = out_degree_from
df_test['out_degree_to'] = out_degree_to
df_test['overall_degree_from'] = overall_degree_from
df_test['overall_degree_to'] = overall_degree_to
df_test['bi_degree_from'] = bi_degree_from
df_test['bi_degree_to'] = bi_degree_to

In [ ]:
# common_friends feature construct(这三个指标，真·废到家了)
in_common_friends = []
out_common_friends = []
bi_common_friends = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    in_common_friends.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(to_list[index], "in")))))
    out_common_friends.append(len(set(g.neighbors(from_list[index], "out")).intersection(set(g.neighbors(to_list[index], "out")))))
    bi_common_friends.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(from_list[index], "out"))).intersection(set(g.neighbors(to_list[index], "in")).intersection(set(g.neighbors(to_list[index], "out"))))))
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
df_test['in_common_friends'] = in_common_friends
df_test['out_common_friends'] = out_common_friends
df_test['bi_common_friends'] = bi_common_friends

In [ ]:
# total_friends feature construct
in_total_friends = []
out_total_friends = []
bi_total_friends = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    in_total_friends.append(len(set(g.neighbors(from_list[index], "in")).union(set(g.neighbors(to_list[index], "in")))))
    out_total_friends.append(len(set(g.neighbors(from_list[index], "out")).union(set(g.neighbors(to_list[index], "out")))))
    bi_total_friends.append(len(set(g.neighbors(from_list[index], "in")).intersection(set(g.neighbors(from_list[index], "out"))).union(set(g.neighbors(to_list[index], "in")).intersection(set(g.neighbors(to_list[index], "out"))))))
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
df_test['in_total_friends'] = in_total_friends
df_test['out_total_friends'] = out_total_friends
df_test['bi_total_friends'] = bi_total_friends

In [ ]:
# transitive_friends feature construct
transitive_friends = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    transitive_friends.append(len(set(g.neighbors(from_list[index], "out")).intersection(set(g.neighbors(to_list[index], "in")))))
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
# preference_attanchment_score feature construct
preference_attanchment_score = []

# for index, row in df.iterrows():
length = len(from_list)
for index in range(length):
    preference_attanchment_score.append(g.degree(from_list[index]) * g.degree(to_list[index]))
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
df_test['transitive_friends'] = transitive_friends
df_test['preference_attanchment_score'] = preference_attanchment_score

In [ ]:
# 以下生成AA特征
import math

adamic_adar = []
# adamic_adar_out_in = []

length = len(from_list)
for index in range(length):
    from_intersection_to = list(set(g.neighbors(from_list[index])).intersection(set(g.neighbors(to_list[index]))))
    log_from_intersection_to = 0
    if len(from_intersection_to) > 0:
        for i in from_intersection_to:
            degree_i = g.degree(i)
            if degree_i > 1: #满足对数的真数大于0，且分母不等于0
                log_from_intersection_to += 1 / math.log(degree_i) # 根据其他文献，这里应该用以e为底的ln

    adamic_adar.append(log_from_intersection_to)
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
import math

# adamic_adar = []
adamic_adar_out_in = []

length = len(from_list)
for index in range(length):
    from_intersection_to = list(set(g.neighbors(from_list[index], "out")).intersection(set(g.neighbors(to_list[index], "in"))))
    log_from_intersection_to = 0
    if len(from_intersection_to) > 0:
        for i in from_intersection_to:
            degree_i = g.degree(i)
            if degree_i > 1: #满足对数的真数大于0，且分母不等于0
                log_from_intersection_to += 1 / math.log(degree_i) # 根据其他文献，这里应该用以e为底的ln

    adamic_adar_out_in.append(log_from_intersection_to)
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
df_test['adamic_adar'] = adamic_adar
df_test['adamic_adar_out_in'] = adamic_adar_out_in

In [ ]:
# friend-measure feature
friend_measure = []

length = len(from_list)
for index in range(length):
    u = from_list[index]
    v = to_list[index]
    u_nei = g.neighbors(u)
    v_nei = g.neighbors(v)
    name_u_nei = []
    name_v_nei = []
    for u_n in u_nei:
        name_u_nei.append(names[u_n])
    for v_n in v_nei:
        name_v_nei.append(names[v_n])
    
    measure_score = 0
    for u_n in name_u_nei:
        for v_n in name_v_nei:
#             if u_n == v_n or (u_n,v_n,1) in road_edges_exist or (v_n,u_n,1) in road_edges_exist:
# are_connected(u,v) Decides whether two given vertices are directly connected. (有向图中，该方法还会考虑方向，只有正向连接才会返回true)
            if u_n == v_n or g.are_connected(u_n, v_n) or g.are_connected(v_n, u_n): 
                measure_score += 1
            
    friend_measure.append(measure_score)
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
# oppsite direction friends features
oppsite_direction_friends = [] # 二值特征，如果（to，from）在链路中，那么from有很大可能关注to

length = len(from_list)
for index in range(length):
    u = from_list[index] # from
    v = to_list[index] # to
    if g.are_connected(v, u):
        oppsite_direction_friends.append(1)
    else:
        oppsite_direction_friends.append(0)
    if (index+1) % 500 == 0:
        print("completed {}".format(index+1))

In [ ]:
df_test['friend_measure'] = friend_measure
df_test['oppsite_direction_friends'] = oppsite_direction_friends

In [ ]:
df_test.to_csv("test_data.csv", index = False, header=True)

In [ ]:
# reshape test set
X_test = df_test.drop(["from", "to"], axis = True)

In [ ]:
# prediction
y_test = RF_now.predict(X_test)

In [ ]:
# construct submission file
Id = [i for i in range(1, 2001)]
data = {"Id":Id, "Predicted":y_test}
df_submit = pd.DataFrame(data)
df_submit.to_csv("submission.csv", index = False, header=True)